# Part 1: Tracking ML Model Predictions

In this tutorial notebook, we'll examine some examples of why we may want to track model predictions, and how this can be done.

In [1]:
from model import SynthesizabilityModel

In [2]:
my_materials = [
    "Li7La3(SnO6)2",
    "Li3(WO3)8",
    "Li4Mn2P4H3O16",
    "Li2Ni(PO3)5",
    "MgV2O4",
]

predictor = SynthesizabilityModel()

In [3]:
for material in my_materials:
    print (
        f"The synthesizability score for {material} is",
        f"{round(predictor.predict_single(material), 2)}."
    )

The synthesizability score for Li7La3(SnO6)2 is 0.05.
The synthesizability score for Li3(WO3)8 is 0.09.
The synthesizability score for Li4Mn2P4H3O16 is 0.41.
The synthesizability score for Li2Ni(PO3)5 is 0.11.
The synthesizability score for MgV2O4 is 0.9.


In [32]:
import json
with open("data/predictions_v1.json", "r") as prediction_file:
    print(json.dumps(
        json.load(prediction_file),
        indent=2
    ))

{
  "synthesizability_predictions": {
    "Li7La3(SnO6)2": 0.15,
    "Li3(WO3)8": 0.39,
    "Li4Mn2P4H3O16": 1.0,
    "Li2Ni(PO3)5": 1.0,
    "MgV2O4": 0.22
  },
  "citation": "https://www.nature.com/articles/s43246-021-00219-x"
}


In [33]:
import pandas                   

In [75]:
files = [
    "data/predictions_v1.json",
    "data/predictions_v2.json",
    "data/predictions_final.json",
]

all_predictions = pandas.concat([           # Concatenate several Series...
        pandas.read_json(f)                 # Read the JSON file to Pandas
        ["synthesizability_predictions"]    # Select the sub-dictionary as a Series
        .rename(f)                          # Rename this Series to the filename
        for f in files                      # Do this for all files in the list
    ], axis=1)                              # Concatenate into a table (not a line)

In [76]:
all_predictions

,data/predictions_v1.json,data/predictions_v2.json,data/predictions_final.json
Li2Ni(PO3)5,1.00,1.00,0.10
Li3(WO3)8,0.39,0.93,0.93
Li4Mn2P4H3O16,1.00,0.99,1.00
Li7La3(SnO6)2,0.15,0.95,0.15
MgV2O4,0.22,0.98,0.22
